In [3]:
import os
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import xml.etree.ElementTree as ET
import shutil

imagenet_train_dir = "data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train"
imagenet_val_dir = "data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val"
imagenet_val_restructured_dir = "data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val_restructured"
annotations_dir = "data/imagenet-object-localization-challenge/ILSVRC/Annotations/CLS-LOC/val"

def restructure_val_dir():
    if os.path.exists(imagenet_val_restructured_dir):
        print("Restructured validation directory already exists.")
        return imagenet_val_restructured_dir

    print("Restructuring validation directory...")

    # Get the class mapping from training directory
    classes = [d.name for d in os.scandir(imagenet_train_dir) if d.is_dir()]

    # Create class directories in the restructured validation directory
    for class_name in classes:
        os.makedirs(os.path.join(imagenet_val_restructured_dir, class_name), exist_ok=True)

    # Function to extract class ID from XML annotation file
    def get_class_from_annotation(xml_path):
        tree = ET.parse(xml_path)
        root = tree.getroot()
        # Get the first object's name (class)
        obj = root.find('object')
        if obj is not None:
            return obj.find('name').text
        return None

    # Process each validation image
    for img_name in os.listdir(imagenet_val_dir):
        if not img_name.endswith('.JPEG'):
            continue

        # Get base name without extension
        base_name = os.path.splitext(img_name)[0]

        # Find corresponding annotation file
        xml_path = os.path.join(annotations_dir, base_name + '.xml')

        if os.path.exists(xml_path):
            class_name = get_class_from_annotation(xml_path)
            if class_name and class_name in classes:
                # Copy image to the appropriate class directory
                src_path = os.path.join(imagenet_val_dir, img_name)
                dst_path = os.path.join(imagenet_val_restructured_dir, class_name, img_name)
                shutil.copy(src_path, dst_path)

    print("Validation directory restructured successfully.")
    return imagenet_val_restructured_dir

# Użycie biblioteki Lightly do transformacji i komponentów SSL
import lightly
from lightly.transforms import SimCLRTransform, DINOTransform, MAETransform


# Wykrycie urządzenia do trenowania (GPU jeśli dostępne)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# --- Przygotowanie zbiorów danych CIFAR10 i CIFAR100 ---
# Transformacje dla trenowania nadzorowanego (baseline i linear probe):
# losowe przycięcie i odbicie (augmentacja) + normalizacja.
train_transform_supervised = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),  # średnie i std dla CIFAR
                         std=(0.2023, 0.1994, 0.2010))
])

# Transformacja dla zbioru testowego (tylko skalowanie do tensoru i normalizacja).
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2023, 0.1994, 0.2010))
])

# Transformacje dla metod samonadzorowanych:
# - Dla SimCLR/MoCo/BYOL: dwie zaugmentowane wersje obrazu.
transform_simclr = SimCLRTransform(input_size=32)   # input_size=32 dla CIFAR
# - Dla DINO: transformacja generująca 2 widoki globalne i 6 lokalnych (domyślnie).
transform_dino = DINOTransform(global_crop_size=32, local_crop_size=16,  # dopasowanie do mniejszych obrazków
                               global_crop_scale=(0.5, 1.0), local_crop_scale=(0.2, 0.5))
# - Dla MAE/SimMIM: jedna widok z losowymi augmentacjami (proste augmentacje).
transform_mae = MAETransform()

# Ładowanie danych CIFAR100 (train i test)
train_dataset_cifar100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True,
                                                       transform=None)  # transform ustawimy później per metoda
test_dataset_cifar100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True,
                                                      transform=test_transform)
# Ładowanie danych CIFAR10 (train i test)
train_dataset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                                     transform=None)
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                                    transform=test_transform)

# Dataloader dla ewaluacji (testy) – tutaj wykorzystamy go do obliczania cech i ewaluacji
test_loader_cifar100 = torch.utils.data.DataLoader(test_dataset_cifar100, batch_size=256, shuffle=False)
test_loader_cifar10 = torch.utils.data.DataLoader(test_dataset_cifar10, batch_size=256, shuffle=False)

# (Opcjonalnie) Przygotowanie zbioru ImageNet-1k, jeżeli dostępny na dysku:
imagenet_train_dir = 'data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train'  # <-- Uwaga: ustawić poprawną ścieżkę jeśli dane dostępne
imagenet_val_dir = 'data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val'
imagenet_train_dataset = None
imagenet_val_dataset = None
if os.path.exists(imagenet_train_dir):
    # Transformacje dla ImageNet: wymiary 224x224 jak w standardowych modelach
    transform_simclr_imagenet = SimCLRTransform(input_size=224)
    transform_dino_imagenet = DINOTransform()  # domyślne parametry dla DINO (224 global, 96 lokal)
    transform_mae_imagenet = MAETransform()
    # transformacje dla baseline i linear eval na ImageNet (przycięcie centralne dla val)
    train_transform_supervised_imnet = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.2, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225))
    ])
    val_transform_imnet = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225))
    ])

    imagenet_val_dir_to_use = restructure_val_dir()

    # Używamy ImageFolder do wczytania danych z katalogu
    imagenet_train_dataset = torchvision.datasets.ImageFolder(root=imagenet_train_dir,
                                                              transform=None)  # transform ustawimy dynamicznie
    imagenet_val_dataset = torchvision.datasets.ImageFolder(root=imagenet_val_dir_to_use,
                                                            transform=val_transform_imnet)
    print("ImageNet datasets prepared.")
else:
    print("ImageNet data not found, skipping ImageNet training in this run.")

Using device: cuda


/home/hussein/pytoniec/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Restructuring validation directory...
Validation directory restructured successfully.
ImageNet datasets prepared.


In [ ]:
from lightly.loss import NTXentLoss, DINOLoss
from lightly.models.modules.heads import SimCLRProjectionHead, DINOProjectionHead, BYOLProjectionHead, BYOLPredictionHead
from lightly.models.utils import update_momentum, deactivate_requires_grad

# 2.1 Trenowanie masked autoencoder (MAE/SimMIM) na zbiorze nieoznaczonym
def pretrain_masked_autoencoder(train_dataset, epochs=20, batch_size=128, lr=1.5e-4):
    """
    Trenuje model typu Masked Autoencoder na podanym zbiorze danych.
    Zwraca wytrenowany encoder (backbone) oraz cały model (encoder+decoder).
    """
    # Ustawiamy transformację dla datasetu (MAETransform przygotowuje random crop i normalizację)
    train_dataset.transform = transform_mae
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    # Tworzymy model - ViT jako encoder, prosta warstwa liniowa jako decoder (SimMIM styl)
    vit = torchvision.models.vit_b_16(pretrained=False)  # ViT-base patch16; dla CIFAR może być nadmiarowy
    # Dostosowanie: zmieniamy rozmiar wejścia patch (CIFAR obraz 32x32, patch 16 -> 2x2 patchy, to za mało)
    # Alternatywnie: powiększamy obrazy CIFAR do 224 wewnątrz transformacji by użyć ViT patch16.
    # (Tutaj zakładamy, że transformacja MAETransform może wewnętrznie robić resize do 224; jeśli nie, warto dodać Resize(224) do transformacji dla CIFAR.)
    # Budujemy model maskowanego autoenkodera:
    class MaskedAutoencoder(nn.Module):
        def __init__(self, vit_encoder):
            super().__init__()
            self.encoder = lightly.models.modules.masked_vision_transformer_torchvision.MaskedVisionTransformerTorchvision(vit=vit_encoder)
            # Decoder: pojedyncza warstwa liniowa mapująca latent do rozmiaru patch (patch_size^2 * 3 kanały)
            hidden_dim = vit_encoder.hidden_dim if hasattr(vit_encoder, 'hidden_dim') else vit_encoder.heads.head.in_features
            patch_size = vit_encoder.patch_size if hasattr(vit_encoder, 'patch_size') else 16
            self.decoder = nn.Linear(hidden_dim, patch_size**2 * 3)
        def forward(self, images):
            # Losowe maskowanie tokenów (75% tokenów maskowane)
            batch_size = images.shape[0]
            seq_len = self.encoder.seq_length  # długość sekwencji patchy (łącznie z tokenem cls jeśli jest)
            # Generowanie maski losowej
            idx_keep, idx_mask = lightly.models.utils.random_token_mask((batch_size, seq_len), mask_ratio=0.75, device=images.device)
            # Encoder dostaje pełny obraz oraz indeksy maskowanych tokenów
            encoded = self.encoder.encode(images=images, idx_mask=idx_mask)
            # Wybieramy tylko reprezentacje niezamaskowanych tokenów (MAE tak robi przed dekoderem)
            encoded_masked = lightly.models.utils.get_at_index(encoded, idx_mask)
            # Dekoder: próba rekonstrukcji oryginalnych patchy dla maskowanych pozycji
            preds = self.decoder(encoded_masked)  # wyniki dla maskowanych patchy
            # Wyznaczamy "ground truth" - faktyczne piksele maskowanych patchy
            patches = lightly.models.utils.patchify(images, patch_size)
            # Uwaga: jeśli ViT dodaje token klas (cls token) to indeksy patchy trzeba zmodyfikować (idx_mask-1)
            target = lightly.models.utils.get_at_index(patches, idx_mask - 1)
            return preds, target

    # Inicjalizacja modelu i ustawienie na urządzenie
    mae_model = MaskedAutoencoder(vit).to(device)
    # Funkcja kosztu - błąd L1 (średni bezwzględny) pomiędzy patchami
    criterion = nn.L1Loss()
    optimizer = torch.optim.AdamW(mae_model.parameters(), lr=lr)
    mae_model.train()
    print(">>> Trenowanie Masked Autoencoder przez {} epok...".format(epochs))
    for epoch in range(epochs):
        total_loss = 0.0
        for (images, _) in train_loader:
            images = images.to(device)
            preds, targets = mae_model(images)        # forward prze maskowany autoenkoder
            loss = criterion(preds, targets)          # oblicz stratę rekonstrukcji
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"[MAE] Epoka {epoch+1}/{epochs}, średni L1 loss: {avg_loss:.4f}")
    # Zwracamy wytrenowany encoder (vit) oraz cały model (encoder+decoder)
    return mae_model.encoder.backbone, mae_model

# 2.2 Trenowanie metody SimCLR (kontrastywna) na zbiorze nieoznaczonym
def pretrain_simclr(train_dataset, epochs=20, batch_size=128, lr=6e-2):
    """
    Trenuje model SimCLR (ResNet18 + projection head) na podanym zbiorze danych.
    Zwraca wytrenowany backbone (ResNet bez klasyfikatora).
    """
    # Ustawienie transformacji dwóch widoków na dataset
    train_dataset.transform = transform_simclr
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    # Tworzymy backbone (ResNet18) i usuwamy ostatnią warstwę klasyfikacji
    resnet = torchvision.models.resnet18(pretrained=False)
    backbone = nn.Sequential(*list(resnet.children())[:-1])  # do przedostatniej warstwy (global avg pool)
    backbone_output_dim = resnet.fc.in_features  # wymiar cech wyjściowych backbone (512 dla ResNet18)
    # Projekcyjna głowica SimCLR: MLP (hidden_dim -> output_dim=128 zazwyczaj)
    projection_head = SimCLRProjectionHead(input_dim=backbone_output_dim, hidden_dim=backbone_output_dim, output_dim=128)
    # Funkcja kosztu NT-Xent (InfoNCE) – Lightly ma implementację. Bez memory bank (tu SimCLR, używamy tylko batch negatywów).
    criterion = NTXentLoss()  # domyślnie temperature=0.5
    optimizer = torch.optim.SGD(list(backbone.parameters()) + list(projection_head.parameters()),
                                lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    backbone.to(device)
    projection_head.to(device)
    backbone.train()
    projection_head.train()
    print(">>> Trenowanie SimCLR przez {} epok...".format(epochs))
    for epoch in range(epochs):
        total_loss = 0.0
        for (views, _) in train_loader:  # views to tupla (x_i, x_j) augmentacji
            x1, x2 = views[0].to(device), views[1].to(device)
            # Obliczamy reprezentacje h dla obu widoków
            h1 = backbone(x1).flatten(start_dim=1)
            h2 = backbone(x2).flatten(start_dim=1)
            # Projekcja z MLP (z) - wektor 128-d do kontrastowania
            z1 = projection_head(h1)
            z2 = projection_head(h2)
            # Oblicz strata NTXentLoss (przyjmuje 2 tensory: pozytywne pary)
            loss = criterion(z1, z2)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"[SimCLR] Epoka {epoch+1}/{epochs}, średni loss: {avg_loss:.4f}")
        scheduler.step()
    return backbone

# 2.2 Trenowanie metody MoCo (Momentum Contrast) na zbiorze nieoznaczonym
def pretrain_moco(train_dataset, epochs=20, batch_size=128, lr=0.06, memory_bank_size=4096):
    """
    Trenuje model MoCo v2 (ResNet18 z encoderem kluczy i kolejką) na podanym zbiorze danych.
    Zwraca wytrenowany backbone (encoder zapytań).
    """
    # Ustawienie transformacji dwóch widoków (tak jak SimCLR)
    train_dataset.transform = transform_simclr
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    # Backbone główny (query encoder) i momentum backbone (key encoder)
    resnet = torchvision.models.resnet18(pretrained=False)
    query_encoder = nn.Sequential(*list(resnet.children())[:-1])
    backbone_output_dim = resnet.fc.in_features  # 512
    # Kopiujemy encoder do key_encoder i zamrażamy gradienty w nim
    key_encoder = copy.deepcopy(query_encoder)
    deactivate_requires_grad(key_encoder)  # wyłączenie obliczania grad dla momentum encodera
    # Projekcyjne głowice dla obu encoderów (MoCo v2 używa MLP jak SimCLR)
    proj_q = SimCLRProjectionHead(input_dim=backbone_output_dim, hidden_dim=backbone_output_dim, output_dim=128)
    proj_k = copy.deepcopy(proj_q)
    deactivate_requires_grad(proj_k)
    # Bufor (queue) na negatywne przykłady
    # Inicjalizacja kolejki losowymi wektorami jednostkowymi
    queue_size = memory_bank_size
    feature_dim = 128
    queue = F.normalize(torch.randn(queue_size, feature_dim, device=device), dim=1)
    queue_ptr = 0  # wskaźnik pozycji do nadpisania w kolejce

    # Funkcja strat InfoNCE (NTXentLoss) z memory bank, żeby wykorzystywać kolejkę jako negatywy
    criterion = NTXentLoss(memory_bank_size=(memory_bank_size, feature_dim))
    optimizer = torch.optim.SGD(list(query_encoder.parameters()) + list(proj_q.parameters()), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    query_encoder.to(device)
    proj_q.to(device)
    key_encoder.to(device)
    proj_k.to(device)
    query_encoder.train(); proj_q.train()
    key_encoder.eval(); proj_k.eval()  # momentum encoder nie trenujemy (ewaluacja - nie aktualizuje BN)
    print(">>> Trenowanie MoCo przez {} epok...".format(epochs))
    momentum = 0.999  # współczynnik momentum do uaktualniania kluczowego encodera
    for epoch in range(epochs):
        total_loss = 0.0
        for (views, _) in train_loader:
            x_q, x_k = views[0].to(device), views[1].to(device)  # pierwsza augmentacja jako query, druga jako key
            # Forward przez query encoder
            q_features = query_encoder(x_q).flatten(start_dim=1)
            q_proj = F.normalize(proj_q(q_features), dim=1)  # znormalizowane z_q (128-dim)
            # Forward przez key encoder (bez gradientu)
            with torch.no_grad():
                # Aktualizujemy parametry key_encoder z momentum (z każdą iteracją upodabniamy do query_encoder)
                update_momentum(query_encoder, key_encoder, m=momentum)
                update_momentum(proj_q, proj_k, m=momentum)
                k_features = key_encoder(x_k).flatten(start_dim=1)
                k_proj = F.normalize(proj_k(k_features), dim=1)  # z_k (target)
            # Obliczamy podobieństwa z query do: pozytyw (k_proj tej samej próbki) oraz negatywy (wszystkie z kolejki)
            # Lightly NTXentLoss z memory bank pozwala to uprościć: przekażemy q_proj i k_proj, gdzie k_proj zostanie dodany do memory bank automatycznie.
            loss = criterion(q_proj, k_proj)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Uaktualnienie kolejki: dodajemy nowe k_proj do queue, zastępując najstarsze
            batch_size_effective = k_proj.shape[0]
            if batch_size_effective <= queue_size:
                # nadpisz najstarsze elementy
                queue[queue_ptr:queue_ptr+batch_size_effective, :] = k_proj.detach()
                queue_ptr = (queue_ptr + batch_size_effective) % queue_size
        avg_loss = total_loss / len(train_loader)
        print(f"[MoCo] Epoka {epoch+1}/{epochs}, średni loss: {avg_loss:.4f}")
        scheduler.step()
    return query_encoder  # zwracamy tylko encoder zapytań (wytrenowany backbone)

# 2.3 Trenowanie metody BYOL (Bootstrap Your Own Latent) na zbiorze nieoznaczonym
def pretrain_byol(train_dataset, epochs=20, batch_size=128, lr=1e-3):
    """
    Trenuje model BYOL (ResNet18 online + target) na podanym zbiorze danych.
    Zwraca wytrenowany backbone (online network).
    """
    # Ustawienie transformacji dwóch widoków (BYOL używa podobnych augmentacji jak SimCLR, ewentualnie dodając solarization, tutaj korzystamy z SimCLRTransform)
    train_dataset.transform = transform_simclr
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    # Definiujemy backbone sieci online i tworzymy kopię do sieci target
    resnet = torchvision.models.resnet18(pretrained=False)
    online_backbone = nn.Sequential(*list(resnet.children())[:-1])
    backbone_output_dim = resnet.fc.in_features  # 512
    target_backbone = copy.deepcopy(online_backbone)
    deactivate_requires_grad(target_backbone)  # sieć target nie ma gradientów
    # Projekcja (MLP) i predykcja dla sieci online, projekcja dla sieci target
    online_proj = BYOLProjectionHead(input_dim=backbone_output_dim, hidden_dim=backbone_output_dim, output_dim=256)
    online_pred = BYOLPredictionHead(input_dim=256, hidden_dim=256, output_dim=256)
    target_proj = copy.deepcopy(online_proj)
    deactivate_requires_grad(target_proj)
    # Optymalizujemy tylko parametry online (backbone, proj, pred)
    optimizer = torch.optim.Adam(list(online_backbone.parameters()) + list(online_proj.parameters()) + list(online_pred.parameters()), lr=lr)
    online_backbone.to(device); online_proj.to(device); online_pred.to(device)
    target_backbone.to(device); target_proj.to(device)
    online_backbone.train(); online_proj.train(); online_pred.train()
    target_backbone.eval(); target_proj.eval()
    momentum = 0.996  # współczynnik momentum do uaktualniania target network
    print(">>> Trenowanie BYOL przez {} epok...".format(epochs))
    for epoch in range(epochs):
        total_loss = 0.0
        for (views, _) in train_loader:
            x_a, x_b = views[0].to(device), views[1].to(device)  # dwie augmentacje
            # Forward przez online network dla obu widoków
            feat_a = online_backbone(x_a).flatten(start_dim=1)
            feat_b = online_backbone(x_b).flatten(start_dim=1)
            proj_a = online_proj(feat_a)
            proj_b = online_proj(feat_b)
            pred_a = online_pred(proj_a)  # predykcja dla a
            pred_b = online_pred(proj_b)  # predykcja dla b
            # Forward przez target network (bez grad)
            with torch.no_grad():
                # momentum update target sieci
                update_momentum(online_backbone, target_backbone, m=momentum)
                update_momentum(online_proj, target_proj, m=momentum)
                # (target_pred nie ma, bo target sieć kończy na projekcji)
                target_feat_a = target_backbone(x_a).flatten(start_dim=1)
                target_feat_b = target_backbone(x_b).flatten(start_dim=1)
                target_proj_a = target_proj(target_feat_a)
                target_proj_b = target_proj(target_feat_b)
            # Normalizacja wektorów projekcji i predykcji
            pred_a_norm = F.normalize(pred_a, dim=1)
            pred_b_norm = F.normalize(pred_b, dim=1)
            target_a_norm = F.normalize(target_proj_b.detach(), dim=1)  # UWAGA: pred_a porównujemy z target z drugiego widoku
            target_b_norm = F.normalize(target_proj_a.detach(), dim=1)
            # Obliczenie straty MSE pomiędzy znormalizowanymi predykcjami online a docelowymi reprezentacjami target
            loss = 2 - 2 * (pred_a_norm * target_a_norm).sum(dim=1).mean() - 2 * (pred_b_norm * target_b_norm).sum(dim=1).mean()
            # (powyższe to równoważnik: loss = MSE(pred_a_norm, target_a_norm) + MSE(pred_b_norm, target_b_norm))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"[BYOL] Epoka {epoch+1}/{epochs}, średni loss: {avg_loss:.4f}")
    return online_backbone

# 2.3 Trenowanie metody DINO (Distillation with No Labels) na zbiorze nieoznaczonym
def pretrain_dino(train_dataset, epochs=20, batch_size=128, lr=1e-3):
    """
    Trenuje model DINO (ResNet18 student + momentum teacher) na podanym zbiorze danych.
    Zwraca wytrenowany backbone (student backbone).
    """
    # Ustawienie transformacji DINO (wiele widoków) na dataset
    train_dataset.transform = transform_dino
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    # Definiujemy backbone (ResNet18) dla studenta i kopiujemy dla nauczyciela
    resnet = torchvision.models.resnet18(pretrained=False)
    student_backbone = nn.Sequential(*list(resnet.children())[:-1])
    backbone_output_dim = resnet.fc.in_features  # 512
    teacher_backbone = copy.deepcopy(student_backbone)
    deactivate_requires_grad(teacher_backbone)
    # Głowice projekcyjne DINO dla studenta i nauczyciela.
    # Używamy DINOProjectionHead: parametry (in_dim, hidden_dim, bottleneck_dim, out_dim, [opcje])
    student_head = DINOProjectionHead(input_dim=backbone_output_dim, hidden_dim=512, bottleneck_dim=256, output_dim=2048, freeze_last_layer=1)
    teacher_head = copy.deepcopy(student_head)
    # teacher_head nie zamrażamy w całości, ale podczas optymalizacji nie będziemy go aktualizować (nie jest w optimizer)
    deactivate_requires_grad(teacher_head)
    # Funkcja kosztu DINO – porównuje wyjścia nauczyciela i studenta (zawiera mechanizm centrowania i temperatury)
    criterion = DINOLoss(output_dim=2048, warmup_teacher_temp_epochs=5)
    # Optimizer tylko dla sieci studenta (backbone + head)
    optimizer = torch.optim.Adam(list(student_backbone.parameters()) + list(student_head.parameters()), lr=lr)
    # Harmonogram zmiany współczynnika momentum (od nieco mniejszego do 1)
    # Będziemy liniowo zwiększać momentum nauczyciela od 0.996 do 1.0 w trakcie epok
    initial_momentum = 0.996
    final_momentum = 1.0
    student_backbone.to(device); student_head.to(device)
    teacher_backbone.to(device); teacher_head.to(device)
    student_backbone.train(); student_head.train()
    teacher_backbone.eval(); teacher_head.eval()
    print(">>> Trenowanie DINO przez {} epok...".format(epochs))
    for epoch in range(epochs):
        total_loss = 0.0
        # Wyznacz wartość momentum dla bieżącej epoki (cosine schedule)
        momentum_val = initial_momentum + (final_momentum - initial_momentum) * (epoch / (epochs - 1))
        for batch in train_loader:
            views = batch[0]  # lista widoków augmentowanych (list length = 2 + n_local_views, domyślnie 8)
            # Uaktualnienie wag nauczyciela (backbone i head) - momentum update przed forward
            update_momentum(student_backbone, teacher_backbone, m=momentum_val)
            update_momentum(student_head, teacher_head, m=momentum_val)
            # Przenieś wszystkie widoki na GPU
            views = [v.to(device) for v in views]
            # DINO: nauczyciel otrzymuje tylko 2 globalne widoki (zwykle 224x224), student wszystkie
            # Zakładamy, że transform_dino generuje 2 pierwsze widoki jako "globalne"
            global_views = views[:2]
            # Obliczenia forward:
            # - Teacher output dla globalnych widoków (zatrzymujemy gradient)
            with torch.no_grad():
                teacher_out = [teacher_head(teacher_backbone(v).flatten(start_dim=1)) for v in global_views]
            # - Student output dla wszystkich widoków
            student_out = [student_head(student_backbone(v).flatten(start_dim=1)) for v in views]
            # Oblicz stratę DINO (porównuje rozkłady wyjściowe teacher vs student)
            loss = criterion(teacher_out, student_out, epoch=epoch)
            optimizer.zero_grad()
            loss.backward()
            # DINO zaleca zablokowanie gradientów ostatniej warstwy projekcyjnej studenta na wczesnych epokach (freeze_last_layer)
            student_head.cancel_last_layer_gradients(current_epoch=epoch)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"[DINO] Epoka {epoch+1}/{epochs}, średni loss: {avg_loss:.4f}")
    return student_backbone

# 2.4 Trenowanie modelu nadzorowanego (supervised) na CIFAR100 (baseline)
def train_supervised_classifier(train_dataset, num_classes=100, epochs=20, batch_size=128, lr=0.1):
    """
    Trenuje model klasyfikacyjny (ResNet18) w sposób nadzorowany na podanym zbiorze (z etykietami).
    Zwraca wytrenowany model (backbone + klasyfikator).
    """
    # Ustawiamy transformacje augmentacyjne dla treningu nadzorowanego
    train_dataset.transform = train_transform_supervised
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    # Tworzymy model ResNet18 z random inicjalizacją (num_classes wyjściowych)
    model = torchvision.models.resnet18(pretrained=False, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    model.train()
    print(">>> Trenowanie modelu nadzorowanego (ResNet18) przez {} epok...".format(epochs))
    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0
        for (images, labels) in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Obliczanie dokładności bieżącej partii (dla monitorowania)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        avg_loss = total_loss / len(train_loader)
        acc = 100.0 * correct / total
        print(f"[Supervised] Epoka {epoch+1}/{epochs}, loss: {avg_loss:.4f}, accuracy: {acc:.2f}%")
        scheduler.step()
    return model

# ---- Wykonanie treningów dla poszczególnych metod na CIFAR100 ----
print("\n=== Rozpoczęcie treningów SSL na CIFAR100 ===")
# # Etap podstawowy:
# backbone_mae_cifar100, mae_model = pretrain_masked_autoencoder(train_dataset_cifar100, epochs=10, batch_size=128)
# backbone_simclr_cifar100 = pretrain_simclr(train_dataset_cifar100, epochs=10, batch_size=128)
# # Etap pośredni (dodatkowo trening MoCo i supervised baseline na CIFAR100):
# backbone_moco_cifar100 = pretrain_moco(train_dataset_cifar100, epochs=10, batch_size=128)
# supervised_model_cifar100 = train_supervised_classifier(train_dataset_cifar100, num_classes=100, epochs=10, batch_size=128)
# backbone_supervised_cifar100 = nn.Sequential(*list(supervised_model_cifar100.children())[:-1])  # ekstrakcja backbone z modelu nadzorowanego
# # Etap zaawansowany (self-distillation metody BYOL i DINO na CIFAR100):
backbone_byol_cifar100 = pretrain_byol(train_dataset_cifar100, epochs=10, batch_size=128)
backbone_dino_cifar100 = pretrain_dino(train_dataset_cifar100, epochs=10, batch_size=128)
print("=== Zakończono pretraining SSL na CIFAR100 ===\n")

# (Opcjonalnie) Trenowanie na ImageNet-1k dla etapów pośredniego/zaawansowanego
if imagenet_train_dataset is not None:
    print("=== Rozpoczęcie treningów SSL na ImageNet-1k (skala demonstracyjna) ===")
    # Ustawiamy odpowiednie transformacje dla ImageNet i tworzymy DataLoader
    imagenet_train_dataset.transform = transform_simclr_imagenet
    imnet_loader = torch.utils.data.DataLoader(imagenet_train_dataset, batch_size=256, shuffle=True, drop_last=True)
    # Dla przykładu trenujemy SimCLR i BYOL na ImageNet kilka epok (w praktyce potrzeba znacznie więcej)
    # backbone_simclr_imnet = pretrain_simclr(imagenet_train_dataset, epochs=2, batch_size=256, lr=0.1)
    # backbone_moco_imnet = pretrain_moco(imagenet_train_dataset, epochs=2, batch_size=256, lr=0.1, memory_bank_size=65536)
    backbone_byol_imnet = pretrain_byol(imagenet_train_dataset, epochs=2, batch_size=256, lr=1e-3)
    backbone_dino_imnet = pretrain_dino(imagenet_train_dataset, epochs=2, batch_size=256, lr=1e-3)
    # Dla ImageNet również można by przeprowadzić linear probing lub ewaluację na CIFAR, ale pomijamy dalsze szczegóły w tym kodzie demonstracyjnym.
    print("=== Zakończono pretraining SSL na ImageNet-1k ===\n")


=== Rozpoczęcie treningów SSL na CIFAR100 ===


/home/hussein/pytoniec/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hussein/pytoniec/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


>>> Trenowanie BYOL przez 10 epok...
[BYOL] Epoka 1/10, średni loss: -1.9407
[BYOL] Epoka 2/10, średni loss: -1.9996


torch.Size([3, 224, 224])